In [1]:
import numpy as np
import matplotlib .pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_excel('sentiment-dataset.xlsx')
dataset

,ID,TITLE,COMMENT,RATING,SENTIMEN
0,1611,Banyak kekurangan,"Tidak seperti di iphone menunya kurang, jadwal...",1,Negative
1,1612,Need update!,Kurang bersahabat ama VoiceOver nih. Semoga ha...,4,Negative
2,1613,Baru seperti ini sudah puas?,"Hampir setahun belum ada update, UI tidak ada ...",1,Negative
3,1614,Crash melulu,Mohon appnya diperbaiki.. Crashnya ga ketolong...,1,Negative
4,1615,Sampah,"Aplikasi sampah, crash mulu,errorny banyak,ga ...",1,Negative
...,...,...,...,...,...
548,2757,Usul,"Aplikasi yg sangat membantu, ada beberapa usul...",3,Positive
549,2758,Mantap!,Keren gan. Update terus agar lebih lengkap.,5,Positive
550,2759,All in one app,Applikasi yang sangat berguna untuk aktifitas ...,5,Positive
551,2760,Lumayan,"Bagus juga, jadi mudah nyari tempat dan simple...",4,Positive


In [3]:
dataset.SENTIMEN = pd.Categorical(dataset.SENTIMEN) 
dataset['id_sentimen'] = dataset.SENTIMEN.cat.codes #save into new column
print(dataset.SENTIMEN)
print(dataset.id_sentimen)

0      Negative
1      Negative
2      Negative
3      Negative
4      Negative
         ...   
548    Positive
549    Positive
550    Positive
551    Positive
552    Positive
Name: SENTIMEN, Length: 553, dtype: category
Categories (3, object): [Negative, Neutral, Positive]
0      0
1      0
2      0
3      0
4      0
      ..
548    2
549    2
550    2
551    2
552    2
Name: id_sentimen, Length: 553, dtype: int8


In [4]:
!pip install Sastrawi

In [5]:
#Cleaning the dataset
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# import Sastrawi package
import re #library for using RegEx
import nltk #library for various NLP toolkits
# nltk_stopword = stopwords.words('indonesian')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stopwords = StopWordRemoverFactory().get_stop_words()

In [7]:
corpus = []
for i in range(0,553):
    review = re.sub('[^a-zA-Z]', ' ', dataset['COMMENT'][i]) #dataset yg ad angka dihapus yg disimpan dari A-Z yang gede kecil
    review = review.lower() #ubah jadi lowercase
    review = review.split() #split kalimat jadi kata-kata e.g.['aplikasi', 'keren', 'banget']
    factory = StemmerFactory() #punya si sastrawi
    stemmer = factory.create_stemmer() #siapin alat buat stemming
    review = [stemmer.stem(word) for word in review if not word in set(stopwords)] #untuk setiap kata di review kalau gak termasuk di stopword akan dicoba stemming
    review = ' '.join(review)
    corpus.append(review)

In [30]:
\

In [24]:
#Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
# Splitting training and testing set
from sklearn.model_selection import train_test_split #it was "sklearn.cross_validation" but now it changed
X = corpus
y = dataset.id_sentimen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.30, random_state=0)

#vect = CountVectorizer()
#vect = CountVectorizer(analyzer='char_wb', ngram_range=(1,1))
vect = TfidfVectorizer(analyzer='word', ngram_range=(1,2))


X_train = vect.fit_transform(X_train).toarray()
X_test = vect.transform(X_test)

In [23]:
# Fitting MultinomialNB Classifier to the training set
#from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# classifier = MultinomialNaiveBayes()
# classifier.fit(X_train, y_train)

# Predicting test results
y_pred = classifier.predict(X_test)
ytest = np.array(y_test)

# f1_score(ytest, y_pred, average='weighted')
print(classification_report(ytest, y_pred))
print(confusion_matrix(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.95      0.72        62
           1       0.00      0.00      0.00        13
           2       0.94      0.66      0.77        91

    accuracy                           0.72       166
   macro avg       0.51      0.54      0.50       166
weighted avg       0.73      0.72      0.69       166

[[59  0  3]
 [12  0  1]
 [31  0 60]]


C:\Users\Ines\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
test = ['Aplikasi gk berguna']
res = classifier.predict(vect.transform(test))
res[0]